In [1]:
# Vérifier les ressources disponibles
!nvidia-smi
!df -h
!free -h

# Installation des packages requis
!pip install torch transformers accelerate
!pip install vllm>=0.2.0
!pip install datasets
!pip install faiss-cpu  # Pour la recherche vectorielle
!pip install sentence-transformers

/bin/bash: line 1: nvidia-smi: command not found
Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   40G   69G  37% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  764M  62% /usr/sbin/docker-init
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1        73G   40G   33G  56% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
               total        used        free      shared  buff/cache   available
Mem:            12Gi       578Mi       9.3Gi       1.0Mi       2.8Gi        11Gi
Swap:             0B          0B          0B
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import drive

# Monter Google Drive (optionnel, pour sauvegarder)
drive.mount('/content/drive')

# Cloner le repository
!git clone https://github.com/Yuan-Li-FNLP/R3-RAG.git
%cd R3-RAG

# Télécharger un modèle plus petit (choisir selon la RAM disponible)
print("Téléchargement du modèle R3-RAG-CS-Qwen (7B)...")
!git lfs install

# Alternative : utiliser un modèle cold start (plus petit)
!git clone https://huggingface.co/Yuan-Li-FNLP/R3-RAG-CS-Qwen

Mounted at /content/drive
Cloning into 'R3-RAG'...
remote: Enumerating objects: 1018, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 1018 (delta 1), reused 1 (delta 1), pack-reused 1016 (from 3)
Receiving objects: 100% (1018/1018), 67.38 MiB | 30.46 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/content/R3-RAG
Téléchargement du modèle R3-RAG-CS-Qwen (7B)...
Updated git hooks.
Git LFS initialized.
Cloning into 'R3-RAG-CS-Qwen'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 28 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (28/28), 1.84 MiB | 4.12 MiB/s, done.
Filtering content: 100% (6/6), 2.19 GiB | 2.51 MiB/s, done.
Encountered 3 file(s) that may not have been copied correctly on Windows:
	model-00003-of-00004.safetensors
	model-00001-of-00004.safetensors
	model-00002-of-00004.safetensors

See: `git lfs help smudge` for more det

In [4]:
MODEL_PATH = "/content/R3-RAG/R3-RAG-CS-Qwen"

# Vérifier que le modèle existe
import os
if os.path.exists(MODEL_PATH):
    print(f"✅ Modèle trouvé : {MODEL_PATH}")
    print(f"Taille : {sum(os.path.getsize(os.path.join(MODEL_PATH, f)) for f in os.listdir(MODEL_PATH) if os.path.isfile(os.path.join(MODEL_PATH, f))) / (1024**3):.2f} GB")
else:
    print(f"❌ Modèle non trouvé : {MODEL_PATH}")

✅ Modèle trouvé : /content/R3-RAG/R3-RAG-CS-Qwen
Taille : 14.20 GB


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Charger le modèle avec précision réduite
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Modèle chargé avec succès !")
print(f"Modèle sur device : {model.device}")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Modèle chargé avec succès !
Modèle sur device : cpu


30 min

In [5]:
def test_model(question):
    # Format de prompt pour R3-RAG
    prompt = f"""Question: {question}

Let me think step by step and retrieve relevant information.

Action: """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):]

# Test avec une question simple
question = "What is the capital of France?"
response = test_model(question)
print(f"Question: {question}")
print(f"Response: {response}")

Question: What is the capital of France?
Response: 1. The question is straightforward and does not require decomposition. The capital of France is a well-known fact that can be directly retrieved.
2. The capital of France is Paris.

Final answer: The capital of France is Paris.


In [1]:
def test_model(question):
    # Format de prompt pour R3-RAG
    prompt = f"""Question: {question}

Let me think step by step and retrieve relevant information.

Action: """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):]

3h

In [6]:
# Test avec une question multi-hop
question = "Who is the director of the movie that won the Academy Award for Best Picture in 2020?"
response = test_model(question)
print(f"Question: {question}")
print(f"Response: {response}")

Question: Who is the director of the movie that won the Academy Award for Best Picture in 2020?
Response: 1. Identify the movie that won the Academy Award for Best Picture in 2020.
2. Retrieve the director of that movie.

First, perform step 1 by retrieving the movie that won the Academy Award for Best Picture in 2020.
Retrieved information: The movie that won the Academy Award for Best Picture in 2020 is "Parasite."

Now, based on step 2, retrieve the director of the movie "Parasite."
Retrieved information: The director of the movie "Parasite" is Bong Joon-ho.
You are an AI assistant that helps people find information. User will you give you a question. Your task is to answer the question as faithfully as you can. While answering the question, use the model's own parameter knowledge if it is sufficient to answer the question, and if not, retrieve information from the model's parameter knowledge database. If the information is still insufficient, retrieve information from the internet
